In [28]:
from enum import StrEnum

In [29]:
Sex = StrEnum('Sex', ('female', 'male'))

class Age:
    def __init__(self, age: int):
        if age < 0:
            raise ValueError('Age cannot be less than zero')
        self.value = age

    def __str__(self):
        last_digit =  self.value % 10
        if last_digit == 1:
            return f'{self.value} год'
        if 1 < last_digit < 5:
            return f'{self.value} года'
        return f'{self.value} лет'            

class Person:
    def __init__(self, fullname: str, age: Age, sex: Sex):
        self.fullname = fullname
        self.age = age
        self.sex = sex

In [30]:
DeviceType = StrEnum('DeviceType', ('mobile', 'laptop', 'desktop', 'tablet'))
Browser = StrEnum('Browser', ('Internet Explorer', 'Chrome', 'Firefox', 'Opera'))

class Region:
    def __init__(self, region: str):
        self.value = region

    def is_established(self) -> bool:
        return self.value != '-'

    def __str__(self):
        if self.is_established():
            return self.value
        return "не установлен"

class Visit:
    def __init__(self, device_type: DeviceType, browser: Browser, region: str):
        self.device_type = device_type
        self.browser = browser
        self.region = region

class Bill:
    def __init__(self, bill: int):
        if bill < 0:
            raise ValueError('Bill cannot be less than zero')
        self.value = bill

    def __str__(self):
        return f'{self.value} у.е.'

class Sale:
    def __init__(self, visit: Visit, bill: Bill):
        self.visit = visit
        self.bill = bill

In [31]:
class Client:
    def __init__(self, person: Person, sale: Sale):
        self.person = person
        self.sale = sale

In [32]:
import csv # todo better use csv.DictReader
class CsvClientRowParser:
    def parse(self, row) -> Client:
        person = self.__parse_row_to_person(row)
        sale = self.__parse_row_sale(row)
        return Client(person, sale)

    def __parse_row_to_person(self, row) -> Person:
        fullname = row[0]
        age = Age(int(row[4]))
        sex = Sex(row[3])
        return Person(fullname, age, sex)

    def __parse_row_visit(self, row) -> Visit:
        device_type = DeviceType(row[1].strip().lower())
        browser = Browser(row[2].strip().lower())
        region = Region(row[6])
        return Visit(device_type, browser, region)
    
    def __parse_row_sale(self, row) -> Sale:
        visit = self.__parse_row_visit(row)
        bill = Bill(int(row[5]))
        return Sale(visit, bill) 

class CsvClientLoader:
    def __init__(self, client_parser: CsvClientRowParser, input_file):
        self.__client_parser = client_parser
        self.input_file = input_file
        self.csv_reader = csv.reader(self.input_file, delimiter=',')
        next(self.csv_reader)

    def __iter__(self):
        return self

    def __next__(self):
        row = next(self.csv_reader)
        return self.__client_parser.parse(row)

In [33]:
class ClientPresenter:
    def present(self, client: Client) -> str:
        person_information = self.__prepare_person_information(client.person)
        sale_information = self.__prepare_sale_information(client)
        visit_information = self.__prepare_visit_information(client.sale.visit)
        return f"{person_information} {sale_information} {visit_information}"

    def __prepare_person_information(self, person: Person) -> str:
        sex = "женского" if person.sex == Sex.female else "мужского"
        return f"Пользователь {person.fullname} {sex}  пола, {person.age}"

    def __prepare_sale_information(self, client: Client) -> str:
        action = f"{"совершила" if client.person.sex == Sex.female else "совершил" } покупку"
        return f"{action} на {client.sale.bill}"

    def __prepare_visit_information(self, visit: Visit) -> str:
        is_mobile_device = visit.device_type in [DeviceType.mobile, DeviceType.tablet]
        device_type_information = "мобильного" if is_mobile_device else "дектопного"

        browser_information = f"{device_type_information} браузера {visit.browser.capitalize()}"

        return f"с {browser_information}. Регион, из которого совершалась покупка: {client.sale.visit.region}"

In [34]:
with open('web_clients_correct.csv', 'r', encoding='utf-8') as input_file:
    with open('web_clients_transform.txt', 'w+', encoding='utf-8') as output_file:
        csv_client_loader = CsvClientLoader(CsvClientRowParser(), input_file)
        client_presenter = ClientPresenter()
    
        for client in csv_client_loader:
            output_file.write(client_presenter.present(client) + '\n')